In [2]:
import sys
import pandas
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import LeakyReLU, Input
from keras.optimizers import SGD, Adam
from sklearn.utils import shuffle
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model


Using TensorFlow backend.


In [3]:
#ONE HOT ENCODES A GIVEN COLUMN
def onehot2(x): return np.array(OneHotEncoder(n_values = 2).fit_transform(x.values.reshape(-1,1)).todense())

In [4]:
#ONE HOT ENCODES A GIVEN COLUMN
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.values.reshape(-1,1)).todense())

In [5]:
#LOAD DATA
train_data = pandas.read_csv("/storage/tanel/child_age_gender/exp/ivectors_2048/train/export.csv", sep=" ")
val_data = pandas.read_csv("/storage/tanel/child_age_gender/exp/ivectors_2048/dev/export.csv", sep=" ")
test_data = pandas.read_csv("/storage/tanel/child_age_gender/exp/ivectors_2048/test/export.csv", sep=" ")

In [5]:
#PREFORMAT

In [6]:
def format(data):
    del data['Unnamed: 605']
    mask = data['AgeGroup'] == 'ag1'
    column_name = 'AgeGroup'
    data.loc[mask, column_name] = 0
    mask = data['AgeGroup'] == 'ag2'
    column_name = 'AgeGroup'
    data.loc[mask, column_name] = 1
    mask = data['AgeGroup'] == 'ag3'
    column_name = 'AgeGroup'
    data.loc[mask, column_name] = 2
    mask = data['Gender'] == 'm'
    column_name = 'Gender'
    data.loc[mask, column_name] = 0
    mask = data['Gender'] == 'f'
    column_name = 'Gender'
    data.loc[mask, column_name] = 1
    return data

train_data = format(train_data)
val_data = format(val_data)
test_data = format(test_data)


In [7]:
train_labels_age_group = onehot(train_data['AgeGroup'])
val_labels_age_group = onehot(val_data['AgeGroup'])
test_labels_age_group = onehot(test_data['AgeGroup'])

train_labels_age = train_data['Age']
val_labels_age = val_data['Age']
test_labels_age = test_data['Age']

train_labels_gender =  onehot(train_data['Gender'])
val_labels_gender = onehot(val_data['Gender'])
test_labels_gender = onehot(test_data['Gender'])

train_i_vectors = train_data.iloc[:, 5:].as_matrix()
val_i_vectors = val_data.iloc[:, 5:].as_matrix()
test_i_vectors = test_data.iloc[:, 5:].as_matrix()


count     70
unique     1
top        0
freq      70
Name: Gender, dtype: int64

In [9]:
#GENDER MODEL
### 0.8829

In [6]:
gender_model = Sequential([
    Dense(4000, activation='relu', input_shape=(600,)),
    Dropout(0.5),
    Dense(2000, activation='relu'),
    Dropout(0.4),
    Dense(2, activation='softmax')])


In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=1, min_lr=0.0001, verbose=1)
gender_model.compile(loss='categorical_crossentropy', optimizer=SGD(0.001), metrics=['accuracy'])


In [8]:
gender_model.fit(x=train_i_vectors,
                    y=train_labels_gender,
                    validation_data=(val_i_vectors, val_labels_gender),
                    epochs=10,
                    batch_size=32)

Train on 17428 samples, validate on 2100 samples
Epoch 1/10
17428/17428 [==============================] - 9s 543us/step - loss: 0.7195 - acc: 0.5716 - val_loss: 0.6608 - val_acc: 0.5814
Epoch 2/10
17428/17428 [==============================] - 6s 362us/step - loss: 0.6421 - acc: 0.6420 - val_loss: 0.6036 - val_acc: 0.7300
Epoch 3/10
11072/17428 [==================>...........] - ETA: 2s - loss: 0.5946 - acc: 0.6898

KeyboardInterrupt: 

In [27]:
gender_model.evaluate(val_i_vectors_gender_19, val_labels_gender_19)

NameError: name 'val_i_vectors_gender_19' is not defined

In [28]:
gender_model.evaluate(test_i_vectors_gender_18, test_labels_gender_18)

70/70 [==============================] - 0s 176us/step


[0.0284677137487701, 1.0]

count    17428.000000
mean        13.638685
std          2.331605
min          9.000000
25%         12.000000
50%         14.000000
75%         15.000000
max         19.000000
Name: Age, dtype: float64

In [ ]:
#AGE GROUP WITH KNOWN GENDER

In [9]:
train_data_males = train_data[train_data['Gender'] == 0]
train_data_females = train_data[train_data['Gender'] == 1]
val_data_males = val_data[val_data['Gender'] == 0]
val_data_females = val_data[val_data['Gender'] == 1]


train_labels_males =  onehot(train_data_males['AgeGroup'])
val_labels_males = onehot(val_data_males['AgeGroup'])

train_i_vectors_males = train_data_males.iloc[:, 5:].as_matrix()
val_i_vectors_males = val_data_males.iloc[:, 5:].as_matrix()

train_labels_females =  onehot(train_data_females['AgeGroup'])
val_labels_females = onehot(val_data_females['AgeGroup'])

train_i_vectors_females = train_data_females.iloc[:, 5:].as_matrix()
val_i_vectors_females = val_data_females.iloc[:, 5:].as_matrix()

In [ ]:
#MALES
### AGE GROUP 0.8345

In [10]:
age_group_model_with_males = Sequential([
    Dense(4000, activation='relu', input_shape=(600,)),
    Dropout(0.8),
    Dense(1000, activation='relu'),
    Dropout(0.4),
    Dense(3, activation='softmax')])

In [11]:
age_group_model_with_males.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


In [12]:
    age_group_model_with_males.fit(x=train_i_vectors_males,
                        y=train_labels_males,
                        validation_data=(val_i_vectors_males, val_labels_males),
                        epochs=10,
                        batch_size=32)

Train on 7140 samples, validate on 1190 samples
Epoch 1/10
7140/7140 [==============================] - 2s 305us/step - loss: 1.1461 - acc: 0.4629 - val_loss: 0.8088 - val_acc: 0.7924
Epoch 2/10
7140/7140 [==============================] - 2s 284us/step - loss: 0.8486 - acc: 0.6141 - val_loss: 0.7158 - val_acc: 0.7824
Epoch 3/10
5600/7140 [======================>.......] - ETA: 0s - loss: 0.6770 - acc: 0.6996

KeyboardInterrupt: 

In [ ]:
č

In [ ]:
#FEMALES
### 0.7363

In [ ]:
age_group_model_with_females = Sequential([
    Dense(4000, activation='relu', input_shape=(600,)),
    Dropout(0.8),
    Dense(1000, activation='relu'),
    Dropout(0.4),
    Dense(3, activation='softmax')])

In [ ]:
age_group_model_with_females.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'], callback=[])


In [ ]:
age_group_model_with_females.fit(x=train_i_vectors_females,
                    y=train_labels_females,
                    validation_data=(val_i_vectors_females, val_labels_females),
                    epochs=10,
                    batch_size=32)

In [ ]:
#MULTI OUTPUT GENDER AGE-GROUP

### GROUP 0.787
### GENDER 0.85

In [ ]:
input_layer = Input(shape=(600,))
x = Dense(4000, activation='relu')(input_layer)
x = Dropout(0.7)(x)
x = Dense(1500, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(300, activation='relu')(x)
x = Dropout(0.3)(x)

output_layer_1 = Dense(3, activation='softmax', name='group_output')(x)
output_layer_2 =  Dense(2, activation='softmax', name='gender_output')(x)

In [ ]:
multi_model = Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])

multi_model.compile(loss={'group_output':'categorical_crossentropy', 'gender_output':'categorical_crossentropy'},
                    optimizer='sgd',
                    metrics={'group_output':'accuracy','gender_output':'accuracy'})

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_group_output_loss', factor=0.8,
                              patience=1, min_lr=0.0001, verbose=1)
multi_model.fit(train_i_vectors,
                [train_labels_age_group, train_labels_gender],
                validation_data=(val_i_vectors, [val_labels_age_group, val_labels_gender]),
                epochs=20,
                )

In [ ]:
#AGE MODEL

In [ ]:
import theano
from keras import backend as K
def age_group_accuracy(y_true, y_pred):
    array = np.array([0]*13 + [1]*2 + [2]*10000000)
    age_to_group = theano.shared(array)
    ages_true = age_to_group[(theano.tensor.iround(y_true))]
    ages_pred = age_to_group[(theano.tensor.iround(y_pred))] 
    return K.mean(K.equal(ages_true, ages_pred), axis=-1)



age_model = Sequential([
   
        Dense(2000, activation = 'relu', input_shape=(600,)),
    Dropout(0.7),
    Dense(600, activation = 'relu', input_shape=(600,)),
    Dropout(0.4),
    Dense(100, activation = 'sigmoid'),
    Dense(1)
])

In [ ]:
age_model.compile(loss='mse',
                  optimizer=SGD(lr=0.01),
                  metrics=[age_group_accuracy])

In [ ]:
age_model.fit(x=train_i_vectors,
              y=train_labels_age,
              validation_data=(val_i_vectors,val_labels_age),
              epochs=50,
              batch_size=32,
              )

In [ ]:
# AGE GROUP MODEL

In [8]:
age_group_model = Sequential([
    Dense(2000, activation='relu', input_shape=(600,)),
    Dropout(0.6),
    Dense(2000, activation='relu'),
    Dropout(0.2),
    Dense(2000, activation='relu'),
    Dropout(0.4),
    Dense(3, activation='softmax')])

In [9]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=1, min_lr=0.0001, verbose=1)
age_group_model.compile(loss='categorical_crossentropy', optimizer=SGD(0.01), metrics=['accuracy'])


In [13]:
age_group_model.fit(x=train_i_vectors,
                    y=train_labels_age_group,
                    validation_data=(val_i_vectors, val_labels_age_group),
                    epochs=2,
                    batch_size=32,
                    callbacks=[reduce_lr])

Train on 17428 samples, validate on 2100 samples
Epoch 1/2
17428/17428 [==============================] - 6s 365us/step - loss: 0.4831 - acc: 0.7978 - val_loss: 0.6891 - val_acc: 0.7729
Epoch 2/2
17428/17428 [==============================] - 6s 363us/step - loss: 0.4425 - acc: 0.8162 - val_loss: 0.6989 - val_acc: 0.7681


2100/2100 [==============================] - 0s 99us/step


[0.65961559846287687, 0.71333333321980064]

In [ ]:
# MULTI OUTPUT AGE GROUP-AGE

In [ ]:
input_layer = Input(shape=(600,))
x = Dense(4000, activation='relu')(input_layer)
x = Dropout(0.7)(x)
x = Dense(300, activation='sigmoid')(x)

output_layer_1 = Dense(3, activation='softmax', name='group_output')(x)
output_layer_2 =  Dense(1, name='age_output')(x)

In [ ]:
multi_model = Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])

multi_model.compile(loss={'group_output':'categorical_crossentropy', 'age_output':'mse'},
                    optimizer=SGD(0.01),
                    metrics={'group_output':'accuracy','age_output':age_group_accuracy})

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_group_output_loss', factor=0.8,
                              patience=1, min_lr=0.0001, verbose=1)
multi_model.fit(train_i_vectors,
                [train_labels_age_group, train_labels_age],
                validation_data=(val_i_vectors, [val_labels_age_group, val_labels_age]),
                epochs=20,
                )

In [ ]:
# MULTI OUTPUT AGE-GENDER

In [98]:
input_layer = Input(shape=(600,))
x = Dense(4000, activation='relu')(input_layer)
x = Dropout(0.7)(x)
x = Dense(500, activation='relu')(input_layer)
x = Dropout(0.4)(x)
x = Dense(200, activation='sigmoid')(x)

output_layer_1 = Dense(2, activation='softmax', name='gender_output')(x)
output_layer_2 =  Dense(1, name='age_output')(x)

INFO (theano.gof.compilelock): Refreshing lock /tmp/hpc_lkpiel/theano.NOBACKUP/compiledir_Linux-3.14.28-x86_64-with-redhat-6.7-Carbon-x86_64-3.5.2-64/lock_dir/lock


In [99]:
multi_model = Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])

multi_model.compile(loss={'gender_output':'categorical_crossentropy', 'age_output':'mse'},
                    optimizer=SGD(0.001),
                    metrics={'gender_output':'accuracy','age_output':age_group_accuracy})

NameError: name 'age_group_accuracy' is not defined

In [ ]:
multi_model.fit(train_i_vectors,
                [train_labels_gender, train_labels_age],
                validation_data=(val_i_vectors, [val_labels_gender, val_labels_age]),
                epochs=100,
                )

In [76]:
lebo = train_data.query('Age == 10 & Gender == 0')

In [79]:
lebo['Gender'].describe()

count     700
unique      1
top         0
freq      700
Name: Gender, dtype: int64

In [10]:
###### AGE MODEL 9
val_u

In [8]:
for i in range(10, 18):
    exec("train_data_gender_%d = train_data[train_data['Age'] == %d]" % (i, i))
    exec("val_data_gender_%d = val_data[val_data['Age'] == %d]" % (i, i))
    exec("test_data_gender_%d = test_data[test_data['Age'] == %d]" % (i, i))
    exec("train_labels_gender_%d = onehot2(train_data_gender_%d['Gender'])" % (i, i))
    exec("val_labels_gender_%d = onehot2(val_data_gender_%d['Gender'])" % (i, i))
    exec("test_labels_gender_%d = onehot2(test_data_gender_%d['Gender'])" % (i, i))
    exec("train_i_vectors_gender_%d = train_data_gender_%d.iloc[:, 5:].as_matrix()" % (i,i))
    exec("val_i_vectors_gender_%d =  val_data_gender_%d.iloc[:, 5:].as_matrix()" % (i,i))
    exec("test_i_vectors_gender_%d =  test_data_gender_%d.iloc[:, 5:].as_matrix()" % (i,i))

    exec("train_data_gender_%d_boys = train_data[train_data['Age'] == %d]" % (i, i))
    exec("val_data_gender_%d_boys = val_data[val_data['Age'] == %d]" % (i, i))
    exec("test_data_gender_%d_boys = test_data[test_data['Age'] == %d]" % (i, i))
    exec("train_labels_gender_%d_boys = onehot2(train_data_gender_%d_boys['Gender'])" % (i, i))
    exec("val_labels_gender_%d_boys = onehot2(val_data_gender_%d_boys['Gender'])" % (i, i))
    exec("test_labels_gender_%d_boys = onehot2(test_data_gender_%d_boys['Gender'])" % (i, i))
    exec("train_i_vectors_gender_%d_boys = train_data_gender_%d_boys.iloc[:, 5:].as_matrix()" % (i, i))
    exec("val_i_vectors_gender_%d_boys =  val_data_gender_%d_boys.iloc[:, 5:].as_matrix()" % (i, i))
    exec("test_i_vectors_gender_%d_boys =  test_data_gender_%d_boys.iloc[:, 5:].as_matrix()" % (i, i))

    
    exec("train_data_gender_%d_girls = train_data[train_data['Age'] == %d]" % (i, i))
    exec("val_data_gender_%d_girls = val_data[val_data['Age'] == %d]" % (i, i))
    exec("test_data_gender_%d_girls = test_data[test_data['Age'] == %d]" % (i, i))
    exec("train_labels_gender_%d_girls = onehot2(train_data_gender_%d_girls['Gender'])" % (i, i))
    exec("val_labels_gender_%d_girls = onehot2(val_data_gender_%d_girls['Gender'])" % (i, i))
    exec("test_labels_gender_%d_girls = onehot2(test_data_gender_%d_girls['Gender'])" % (i, i))
    exec("train_i_vectors_gender_%d_girls = train_data_gender_%d_girls.iloc[:, 5:].as_matrix()" % (i, i))
    exec("val_i_vectors_gender_%d_girls =  val_data_gender_%d_girls.iloc[:, 5:].as_matrix()" % (i, i))
    exec("test_i_vectors_gender_%d_girls =  test_data_gender_%d_girls.iloc[:, 5:].as_matrix()" % (i, i))


In [39]:
train_labels_gender_9

,Utterance,Speaker,Gender,Age,AgeGroup,ivec1,ivec2,ivec3,ivec4,ivec5,...,ivec591,ivec592,ivec593,ivec594,ivec595,ivec596,ivec597,ivec598,ivec599,ivec600
0,143_10001143_100_ch1,143_1,0,9,0,-0.418398,-0.938296,0.669691,-1.463043,0.453503,...,0.128172,0.934197,0.765379,-0.335819,0.070354,0.296477,1.252707,1.078482,-0.801526,0.871624
1,143_10001143_10_ch1,143_1,0,9,0,-0.633367,-0.525246,0.424735,-1.210979,1.081761,...,-0.630734,-0.029553,-0.920571,0.509557,-0.864417,-0.295759,0.456390,0.789470,0.396610,0.723979
2,143_10001143_129_ch1,143_1,0,9,0,-1.066284,0.357609,-0.992035,-2.905210,0.573784,...,-0.894630,0.442736,0.312838,0.649240,-0.143712,0.121022,-0.974557,0.138829,-0.163264,-0.292201
3,143_10001143_1432_ch1,143_1,0,9,0,-0.592274,-0.974527,0.520979,-0.997169,0.200358,...,0.966990,-1.037654,-0.314220,0.070033,1.538839,0.406294,-0.334310,-0.633877,0.171371,0.854166
4,143_10001143_1433_ch1,143_1,0,9,0,-1.050412,-0.750296,0.077094,-1.177778,0.081897,...,0.109936,0.118326,1.261557,0.046693,-0.327969,1.091086,-0.039490,0.975986,1.139612,-0.202748
5,143_10001143_1434_ch1,143_1,0,9,0,-0.627047,-0.383151,-0.159873,-1.425629,-0.476528,...,-0.311192,0.765314,0.593452,0.092819,0.672037,0.594454,0.236099,0.895553,0.071543,0.121591
6,143_10001143_1435_ch1,143_1,0,9,0,-1.078822,-0.456977,0.876714,-0.795689,0.072347,...,-1.540425,0.654016,0.735474,1.460004,0.566635,0.684433,-0.643174,0.192285,-0.072796,0.381613
7,143_10001143_157_ch1,143_1,0,9,0,-0.300843,-0.825967,0.687509,-1.794576,-0.476436,...,-0.761839,-0.776795,0.038089,-0.631255,-0.029913,-0.126166,1.100268,-0.858795,0.039926,-1.009529
8,143_10001143_158_ch1,143_1,0,9,0,-1.177312,-0.870501,0.896570,-1.265312,-0.064263,...,-0.904038,-0.783829,-0.612597,0.058214,-0.657690,0.468953,0.783177,0.528017,0.952097,0.158702
9,143_10001143_1602_ch1,143_1,0,9,0,-1.086748,-0.659327,0.257502,-2.165888,0.320460,...,1.509905,-0.202921,0.492288,-0.447707,-1.585893,0.333516,1.199462,-0.947476,0.438035,0.547364


In [19]:
print (print9)

NameError: name 'print9' is not defined